In [1]:
# Making imports convenient
import sys
import os
PATH=os.getcwd().split('/notebooks')[0]
sys.path.insert(1, PATH)

import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from datasets import load_dataset, Dataset,concatenate_datasets
import transformers
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold

from transformers import AutoTokenizer, DataCollatorWithPadding,AutoModelForSequenceClassification,AdamW,get_scheduler,TrainingArguments,Trainer,EarlyStoppingCallback

from src.utils.myutils import *
import yaml
from tqdm import tqdm

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

CS_DATA_PATH = PATH + '/data/CS/processed/BABE/train.csv'
CONFIG_PATH = PATH + '/src/utils/config.yaml'

BATCH_SIZE = 64
transformers.utils.logging.set_verbosity_error()

### BABE train_test split (SKIP IF DONE)

In [2]:
babe = load_dataset("csv", data_files=PATH + '/data/CS/raw/BABE/SG2.csv')['train']

Using custom data configuration default-41acc90be2294f89
Reusing dataset csv (/home/horyctom/.cache/huggingface/datasets/csv/default-41acc90be2294f89/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff)


In [3]:
babe = babe.train_test_split(0.15,seed=42)

Loading cached split indices for dataset at /home/horyctom/.cache/huggingface/datasets/csv/default-41acc90be2294f89/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff/cache-2e89d302da86ff73.arrow and /home/horyctom/.cache/huggingface/datasets/csv/default-41acc90be2294f89/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff/cache-7a7ea8491428011d.arrow


In [4]:
babe['train'].to_csv(PATH + '/data/CS/processed/BABE/train.csv',index=False)
babe['test'].to_csv(PATH + '/data/CS/processed/BABE/test.csv',index=False) #THIS IS FOR THE FINAL MODEL SELECTED,TUNED

125614

## Load data

In [2]:
data = load_dataset('csv',data_files = CS_DATA_PATH)['train']
data

Using custom data configuration default-f28f8af5b44ab214
Reusing dataset csv (/home/horyctom/.cache/huggingface/datasets/csv/default-f28f8af5b44ab214/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff)


Dataset({
    features: ['sentence', 'label'],
    num_rows: 3122
})

In [3]:
with open(CONFIG_PATH) as f:
    config_data = yaml.load(f, Loader=yaml.FullLoader)

## Training

In [4]:
def compute_metrics_eval(eval_preds):
    metric = load_metric("f1")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [5]:
skfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [6]:
training_args = TrainingArguments(
    output_dir = './',
    num_train_epochs=3,
    per_device_train_batch_size=BATCH_SIZE,  
    logging_steps=50,
    disable_tqdm = False,
    save_total_limit=2,
    weight_decay=0.1,
    learning_rate=5e-5)

### Cross-Val all models

In [12]:
model_scores = {}

for model_name in tqdm(config_data['models']):
    scores = []
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False,padding=True)
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    token_full = preprocess_data(data,tokenizer,'sentence')

    print("Running 5-fold CV on model: ",model_name,"...")
    for train_index, val_index in skfold.split(token_full['input_ids'],token_full['label']):

        token_train = Dataset.from_dict(token_full[train_index])
        token_valid = Dataset.from_dict(token_full[val_index])


        model = AutoModelForSequenceClassification.from_pretrained(model_name,num_labels=2);
        model.to(device)
        trainer = Trainer(model,training_args,train_dataset=token_train,data_collator=data_collator,tokenizer=tokenizer)
        trainer.train()

        #evaluation
        eval_dataloader = DataLoader(token_valid, batch_size=BATCH_SIZE, collate_fn=data_collator)
        scores.append(compute_metrics(model,device,eval_dataloader)['f1'])
        
    print("Done.")
    model_scores[model_name] = scores
    

with open("./results.txt",'w') as f:
    f.write(json.dumps(model_scores))


  0%|          | 0/6 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/675 [00:00<?, ?B/s]

Loading cached processed dataset at /home/horyctom/.cache/huggingface/datasets/csv/default-f28f8af5b44ab214/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff/cache-53a71358952df90f.arrow
/home/horyctom/venv/lib/python3.8/site-packages/datasets/formatting/formatting.py:167: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(array, copy=False, **self.np_array_kwargs)
/home/horyctom/venv/lib/python3.8/site-packages/datasets/formatting/torch_formatter.py:44: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before conv

Running 5-fold CV on model:  UWB-AIR/Czert-B-base-cased ...


Downloading:   0%|          | 0.00/441M [00:00<?, ?B/s]

...

Step,Training Loss
50,0.553900
100,0.376700




Training completed. Do not forget to share your model on huggingface.co/models =)


/home/horyctom/venv/lib/python3.8/site-packages/datasets/formatting/formatting.py:167: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(array, copy=False, **self.np_array_kwargs)
loading configuration file https://huggingface.co/DeepPavlov/bert-base-bg-cs-pl-ru-cased/resolve/main/config.json from cache at /home/horyctom/.cache/huggingface/transformers/969927d8dc3ef116510e05257d0982836522039e0a62148f115b2cd116f6dafb.e8f15c5aad2f4653e46ceeba0bb32c02a629d106a902c964bce60523d290ac8f
Model config BertConfig {
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  

Step,Training Loss
50,0.566300
100,0.380300




Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 6/6 [24:33<00:00, 245.59s/it]

Done.


In [13]:
model_scores

{'UWB-AIR/Czert-B-base-cased': [0.7856,
  0.776,
  0.7628205128205128,
  0.780448717948718,
  0.7692307692307693],
 'ufal/robeczech-base': [0.7776,
  0.7648,
  0.7772435897435898,
  0.7788461538461539,
  0.7708333333333333],
 'bert-base-multilingual-cased': [0.7584000000000001,
  0.7088,
  0.7035256410256411,
  0.7644230769230768,
  0.719551282051282],
 'fav-kky/FERNET-C5': [0.7968,
  0.7824,
  0.7612179487179487,
  0.7788461538461539,
  0.7580128205128205],
 'fav-kky/FERNET-News': [0.5104,
  0.7312,
  0.49038461538461536,
  0.7339743589743589,
  0.7676282051282052],
 'DeepPavlov/bert-base-bg-cs-pl-ru-cased': [0.7776,
  0.7504,
  0.735576923076923,
  0.780448717948718,
  0.7467948717948718]}

In [14]:
for model in config_data['models']:
    print(model,"F1 score:",np.mean(model_scores[model]))

UWB-AIR/Czert-B-base-cased F1 score: 0.77482
ufal/robeczech-base F1 score: 0.7738646153846153
bert-base-multilingual-cased F1 score: 0.7309399999999999
fav-kky/FERNET-C5 F1 score: 0.7754553846153845
fav-kky/FERNET-News F1 score: 0.6467174358974359
DeepPavlov/bert-base-bg-cs-pl-ru-cased F1 score: 0.7581641025641026


In [10]:
import json